# 🖼️ 图文检索系统 - CLIP + FAISS

本Notebook实现了一个完整的图文检索系统：
- 使用 **CLIP** 模型对图片和文本进行编码
- 使用 **FAISS** 建立向量索引，实现高效检索
- 支持 **文本检索图片** 和 **图片检索文本**

## 功能特点
1. 文本检索图片：输入描述，返回最相关的图片
2. 图片检索文本：输入图片，返回最相关的描述
3. 以图搜图：输入图片，返回相似的图片

## 1. 环境安装

In [ ]:
# 安装依赖
!pip install -q ftfy regex tqdm
!pip install -q git+https://github.com/openai/CLIP.git
!pip install -q faiss-cpu
!pip install -q pycocotools

In [ ]:
# 导入库
import os
import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import faiss
import clip
import requests
import zipfile
from pycocotools.coco import COCO
import pickle
from typing import List, Tuple

print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 定义图文检索类

In [ ]:
class ImageTextRetrieval:
    def __init__(self, device=None, clip_model="ViT-B/32", index_type="flat"):
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = torch.device(device)
        
        print(f"🖥️ 使用设备: {self.device}")
        
        self.clip_model_name = clip_model
        self.index_type = index_type
        
        self.model = None
        self.preprocess = None
        self.image_index = None
        self.text_index = None
        self.image_embeddings = None
        self.text_embeddings = None
        self.image_paths = []
        self.texts = []
        
        self._load_clip_model()
    
    def _load_clip_model(self):
        print(f"📥 加载CLIP模型: {self.clip_model_name}")
        self.model, self.preprocess = clip.load(self.clip_model_name, device=self.device)
        self.model.eval()
        print("✅ CLIP模型加载完成")
    
    def encode_images(self, image_paths, batch_size=32):
        print(f"🖼️ 编码 {len(image_paths)} 张图片...")
        embeddings = []
        
        for i in tqdm(range(0, len(image_paths), batch_size)):
            batch_paths = image_paths[i:i + batch_size]
            batch_images = []
            
            for path in batch_paths:
                try:
                    image = Image.open(path).convert('RGB')
                    image = self.preprocess(image)
                    batch_images.append(image)
                except Exception as e:
                    batch_images.append(torch.zeros(3, 224, 224))
            
            batch_images = torch.stack(batch_images).to(self.device)
            
            with torch.no_grad():
                batch_embeddings = self.model.encode_image(batch_images)
                batch_embeddings = F.normalize(batch_embeddings, dim=-1)
            
            embeddings.append(batch_embeddings.cpu().numpy())
        
        self.image_embeddings = np.vstack(embeddings).astype('float32')
        self.image_paths = image_paths
        print(f"✅ 图片编码完成，形状: {self.image_embeddings.shape}")
        return self.image_embeddings
    
    def encode_texts(self, texts, batch_size=32):
        print(f"📝 编码 {len(texts)} 条文本...")
        embeddings = []
        
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i + batch_size]
            
            with torch.no_grad():
                text_tokens = clip.tokenize(batch_texts, truncate=True).to(self.device)
                batch_embeddings = self.model.encode_text(text_tokens)
                batch_embeddings = F.normalize(batch_embeddings, dim=-1)
            
            embeddings.append(batch_embeddings.cpu().numpy())
        
        self.text_embeddings = np.vstack(embeddings).astype('float32')
        self.texts = texts
        print(f"✅ 文本编码完成，形状: {self.text_embeddings.shape}")
        return self.text_embeddings
    
    def build_index(self, embeddings, nlist=100, nprobe=10):
        d = embeddings.shape[1]
        
        if self.index_type == "flat":
            index = faiss.IndexFlatIP(d)
        elif self.index_type == "ivf":
            quantizer = faiss.IndexFlatIP(d)
            index = faiss.IndexIVFFlat(quantizer, d, nlist)
            index.train(embeddings)
            index.nprobe = nprobe
        elif self.index_type == "hnsw":
            M = 32
            index = faiss.IndexHNSWFlat(d, M)
            index.hnsw.efSearch = 64
        else:
            raise ValueError(f"不支持的索引类型: {self.index_type}")
        
        index.add(embeddings)
        print(f"✅ 索引构建完成，包含 {index.ntotal} 个向量")
        return index
    
    def build_image_index(self, nlist=100):
        if self.image_embeddings is None:
            raise ValueError("请先编码图片")
        self.image_index = self.build_index(self.image_embeddings, nlist)
        return self.image_index
    
    def build_text_index(self, nlist=100):
        if self.text_embeddings is None:
            raise ValueError("请先编码文本")
        self.text_index = self.build_index(self.text_embeddings, nlist)
        return self.text_index
    
    def search_text_to_image(self, query, k=5):
        with torch.no_grad():
            text_token = clip.tokenize([query], truncate=True).to(self.device)
            query_embedding = self.model.encode_text(text_token)
            query_embedding = F.normalize(query_embedding, dim=-1)
            query_embedding = query_embedding.cpu().numpy().astype('float32')
        
        D, I = self.image_index.search(query_embedding, k)
        
        results = []
        for idx, score in zip(I[0], D[0]):
            if idx < len(self.image_paths):
                results.append((self.image_paths[idx], float(score)))
        
        return results
    
    def search_image_to_text(self, image_path, k=5):
        image = Image.open(image_path).convert('RGB')
        image = self.preprocess(image).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            query_embedding = self.model.encode_image(image)
            query_embedding = F.normalize(query_embedding, dim=-1)
            query_embedding = query_embedding.cpu().numpy().astype('float32')
        
        D, I = self.text_index.search(query_embedding, k)
        
        results = []
        for idx, score in zip(I[0], D[0]):
            if idx < len(self.texts):
                results.append((self.texts[idx], float(score)))
        
        return results
    
    def search_image_to_image(self, image_path, k=5):
        image = Image.open(image_path).convert('RGB')
        image = self.preprocess(image).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            query_embedding = self.model.encode_image(image)
            query_embedding = F.normalize(query_embedding, dim=-1)
            query_embedding = query_embedding.cpu().numpy().astype('float32')
        
        D, I = self.image_index.search(query_embedding, k + 1)
        
        results = []
        for idx, score in zip(I[0], D[0]):
            if idx < len(self.image_paths) and self.image_paths[idx] != image_path:
                results.append((self.image_paths[idx], float(score)))
        
        return results[:k]
    
    def visualize_text_to_image(self, query, results):
        n = len(results)
        fig, axes = plt.subplots(1, n + 1, figsize=(4 * (n + 1), 4))
        
        axes[0].text(0.5, 0.5, query, ha='center', va='center', fontsize=12, wrap=True,
                    bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))
        axes[0].set_title("🔍 查询文本", fontsize=12)
        axes[0].axis('off')
        
        for i, (img_path, score) in enumerate(results):
            try:
                img = Image.open(img_path).convert('RGB')
                axes[i + 1].imshow(img)
                axes[i + 1].set_title(f"相似度: {score:.4f}", fontsize=10)
            except:
                axes[i + 1].text(0.5, 0.5, "加载失败", ha='center', va='center')
            axes[i + 1].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    def visualize_image_to_text(self, image_path, results):
        n = len(results)
        fig, axes = plt.subplots(1, n + 1, figsize=(4 * (n + 1), 4))
        
        query_img = Image.open(image_path).convert('RGB')
        axes[0].imshow(query_img)
        axes[0].set_title("🔍 查询图片", fontsize=12)
        axes[0].axis('off')
        
        for i, (text, score) in enumerate(results):
            axes[i + 1].text(0.5, 0.5, text, ha='center', va='center', fontsize=9, wrap=True,
                           bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
            axes[i + 1].set_title(f"相似度: {score:.4f}", fontsize=10)
            axes[i + 1].axis('off')
        
        plt.tight_layout()
        plt.show()

## 3. 下载COCO数据集

In [ ]:
def download_coco_dataset(sample_size=1000, save_dir="./coco_data"):
    os.makedirs(save_dir, exist_ok=True)
    
    print("📥 下载COCO标注文件...")
    ann_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
    ann_zip = os.path.join(save_dir, "annotations.zip")
    
    if not os.path.exists(ann_zip):
        response = requests.get(ann_url, stream=True)
        with open(ann_zip, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        with zipfile.ZipFile(ann_zip, 'r') as zip_ref:
            zip_ref.extractall(save_dir)
    
    ann_file = os.path.join(save_dir, "annotations", "captions_val2017.json")
    coco = COCO(ann_file)
    
    img_dir = os.path.join(save_dir, "images")
    os.makedirs(img_dir, exist_ok=True)
    
    img_ids = list(coco.imgs.keys())[:sample_size]
    
    image_paths = []
    texts = []
    
    print(f"📥 下载 {len(img_ids)} 张图片...")
    for img_id in tqdm(img_ids):
        img_info = coco.loadImgs(img_id)[0]
        img_url = img_info['coco_url']
        img_path = os.path.join(img_dir, img_info['file_name'])
        
        if not os.path.exists(img_path):
            try:
                response = requests.get(img_url, stream=True, timeout=10)
                with open(img_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
            except Exception as e:
                continue
        
        image_paths.append(img_path)
        
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)
        if anns:
            texts.append(anns[0]['caption'])
    
    print(f"✅ 数据集准备完成: {len(image_paths)} 张图片, {len(texts)} 条文本")
    return image_paths, texts

In [ ]:
# 下载COCO数据集（可以调整sample_size）
SAMPLE_SIZE = 500  # 可根据需要调整
image_paths, texts = download_coco_dataset(sample_size=SAMPLE_SIZE)

## 4. 构建索引

In [ ]:
# 初始化检索系统
retrieval = ImageTextRetrieval(device="cuda", index_type="flat")

In [ ]:
# 编码图片和文本
retrieval.encode_images(image_paths, batch_size=64)
retrieval.encode_texts(texts, batch_size=64)

In [ ]:
# 构建索引
retrieval.build_image_index()
retrieval.build_text_index()

## 5. 文本检索图片演示

In [ ]:
# 测试查询
test_queries = [
    "a dog playing in the park",
    "people walking on the street",
    "a car on the road",
    "a beautiful sunset over the ocean",
    "children playing with toys"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"🔍 查询: {query}")
    print('='*60)
    
    results = retrieval.search_text_to_image(query, k=5)
    
    for i, (path, score) in enumerate(results):
        print(f"  {i+1}. {os.path.basename(path)} (相似度: {score:.4f})")
    
    retrieval.visualize_text_to_image(query, results)

## 6. 图片检索文本演示

In [ ]:
# 随机选择几张图片进行测试
import random
test_images = random.sample(image_paths, 3)

for img_path in test_images:
    print(f"\n{'='*60}")
    print(f"🖼️ 查询图片: {os.path.basename(img_path)}")
    print('='*60)
    
    results = retrieval.search_image_to_text(img_path, k=3)
    
    for i, (text, score) in enumerate(results):
        print(f"  {i+1}. {text} (相似度: {score:.4f})")
    
    retrieval.visualize_image_to_text(img_path, results)

## 7. 以图搜图演示

In [ ]:
# 以图搜图
test_img = random.choice(image_paths)

print(f"\n{'='*60}")
print(f"🖼️ 查询图片: {os.path.basename(test_img)}")
print('='*60)

results = retrieval.search_image_to_image(test_img, k=5)

# 可视化
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# 显示查询图片
query_img = Image.open(test_img).convert('RGB')
axes[0, 0].imshow(query_img)
axes[0, 0].set_title("查询图片", fontsize=12, color='red')
axes[0, 0].axis('off')

# 显示结果
for i, (path, score) in enumerate(results[:5]):
    row, col = (i + 1) // 3, (i + 1) % 3
    img = Image.open(path).convert('RGB')
    axes[row, col].imshow(img)
    axes[row, col].set_title(f"相似度: {score:.4f}", fontsize=10)
    axes[row, col].axis('off')

# 隐藏空白子图
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

## 8. 自定义查询测试

In [ ]:
# 在这里输入你自己的查询
my_query = "a person riding a bicycle"  # 修改这里

print(f"🔍 查询: {my_query}")
results = retrieval.search_text_to_image(my_query, k=5)

for i, (path, score) in enumerate(results):
    print(f"  {i+1}. {os.path.basename(path)} (相似度: {score:.4f})")

retrieval.visualize_text_to_image(my_query, results)

## 9. 性能分析

In [ ]:
import time

# 测试检索速度
query = "a dog running in the park"

start_time = time.time()
for _ in range(100):
    results = retrieval.search_text_to_image(query, k=5)
end_time = time.time()

avg_time = (end_time - start_time) / 100 * 1000
print(f"📊 平均检索时间: {avg_time:.2f} ms")
print(f"📊 索引大小: {retrieval.image_index.ntotal} 个向量")
print(f"📊 向量维度: {retrieval.image_embeddings.shape[1]}")

## 10. 总结

### 本系统实现了：
1. **文本检索图片** - 输入自然语言描述，返回相关图片
2. **图片检索文本** - 输入图片，返回相关描述
3. **以图搜图** - 输入图片，返回相似图片

### 技术栈：
- **CLIP**: OpenAI的多模态预训练模型，将图片和文本映射到同一向量空间
- **FAISS**: Facebook的高效向量检索库，支持大规模相似度搜索

### 扩展方向：
1. 使用更大的数据集
2. 尝试不同的CLIP模型（如ViT-L/14）
3. 使用FAISS的IVF或HNSW索引加速大规模检索
4. 添加重排序模块提升精度